## Docker Networking
docker has three types of networks:
* docker run ubuntu # creates a new container and run it on the default network bridge
* docker run --network=host ubuntu # run the container on the host network
* docker run --network=none ubuntu # run the container with no network

**list networks**
docker network ls

**create a new network**
docker network create mynetwork \
--driver bridge \
--subnet 182.18.0.0/16 \
custom-isolated-network


**inspect a network**
docker network inspect mynetwork
docker run --network=mynetwork ubuntu # run the container on the custom network

docker inspect container_id | grep IPAddress # get the ip address of a container

**Emdeded DNS server**
Docker has an embedded DNS server that provides DNS resolution to all containers on the same network. The DNS server is reachable at 
mysql.connect(mysqldb, user='root', password='password') # connect to mysql container using the container name

In [12]:
%%bash 
# create a new network
docker network create  \
--driver bridge \
--subnet 182.18.0.0/16 \
custom-isolated-network

bc0f9914d5368276195df881045ff714e5467e4e69b39148ccf17daeab96384c


In [13]:
%%bash
# list networks
docker network ls

NETWORK ID     NAME                      DRIVER    SCOPE
0cc715522748   /                         bridge    local
a07afdee07eb   bridge                    bridge    local
bc0f9914d536   custom-isolated-network   bridge    local
aa04e1c37bd8   docker-nginx_backend      bridge    local
c19ac9c377e7   docker-nginx_frontend     bridge    local
337b85f1d2a8   host                      host      local
dece7ad46d20   none                      null      local


In [14]:
%%bash 
docker network inspect custom-isolated-network

[
    {
        "Name": "custom-isolated-network",
        "Id": "bc0f9914d5368276195df881045ff714e5467e4e69b39148ccf17daeab96384c",
        "Created": "2024-03-25T12:39:11.52762276Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "182.18.0.0/16"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [17]:
%%bash 
docker run -d -p 8080:80 --name webserver --network custom-isolated-network webapp-color

89b8bc01d5bd1a89fdca23e2c80700e92d50d1e6a384a0da52835c1daff1cf61


In [18]:
%%bash 
#We just ran a container named webserver. Identify the network it is attached to.
docker inspect webserver | grep NetworkMode

            "NetworkMode": "custom-isolated-network",


In [19]:
%%bash
# What is the subnet configured on custom-isolated-network network?
docker network inspect custom-isolated-network | grep Subnet

                    "Subnet": "182.18.0.0/16"


In [28]:
%%bash
# remove the network 
docker network rm custom-isolated-network

custom-isolated-network


In [29]:
%%bash 
# Create a new network named wp-mysql-network using the bridge driver. 
# Allocate subnet 182.18.0.1/24. Configure Gateway 182.18.0.1
docker network create --driver bridge --subnet 182.18.0.1/24 --gateway 182.18.0.1 wp-mysql-network

fcb01abb846971b6d6bf4a7ee9ea800770d33f31f0b57b13fd7c99231e9f7b84


In [31]:
%%bash 
# Deploy a mysql database using the mysql:5.6 image
#  and name it mysql-db.
#  Attach it to the newly created network wp-mysql-network
#Set the database password to use db_pass123. The environment variable to set is MYSQL_ROOT_PASSWORD.
docker run -d -e MYSQL_ROOT_PASSWORD=db_pass123 --name mysql-db --network wp-mysql-network mysql:5.6

52016684c4202561c9c7c6a4812611d32bcd8c3d1d5faa5d8b7d6f0caf459bb4


In [44]:
%%bash 
#Deploy a web application named webapp using the kodekloud/simple-webapp-mysql image.
#  Expose the port to 38080 on the host.

#The application makes use of two environment variable:
#1: DB_Host with the value mysql-db.
#2: DB_Password with the value db_pass123.
#Make sure to attach it to the newly created network called wp-mysql-network.
#Also make sure to link the MySQL and the webapp container.
docker run --network=wp-mysql-network -e DB_Host=mysql-db \
           -e DB_Password=db_pass123  -p 38080:8080 --name webapp \
           --link mysql-db:mysql-db -d kodekloud/simple-webapp-mysql

6f6c71995750bc33c4fcfb3cf1a23427f45100e3e9d205f9e80f89735b237df5
